In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"


# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted a

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
vine_table = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
vine_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [5]:
####################### Deliverable 2 ###########################

#Retrieve all the rows where the total_votes count is equal to or greater than 20

#review_id_df.filter(review_id_df["review_id"] == 'R1QX6706ZWJ1P5').show()

vine_table = vine_table.filter(vine_table["total_votes"] >= '20')
vine_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [6]:
#retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.


vine_table = vine_table.filter((vine_table["helpful_votes"]/vine_table["total_votes"]) >= '0.5')

#filtered_df_new = filtered_df.filter(filtered_df["helpful_votes"] == '39')
vine_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [8]:
#retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'

vine_table_paid = vine_table.filter(vine_table['vine'] == 'Y')
vine_table_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...| 2014-01-13|
|         US|   50929595|R1RZ4JZO8DAP7H|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App

In [9]:
#retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
vine_table_unpaid = vine_table.filter(vine_table['vine'] == 'N')
vine_table_unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [20]:
#Determine the total number of reviews, the number of 5-star reviews, 
#and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
import pandas as pd

# Total number of reviews
paid_total_reviews = vine_table_paid.select("star_rating").count()
unpaid_total_reviews = vine_table_unpaid.select("star_rating").count()

#Number of 5-star reviews
paid_5_star = vine_table_paid[vine_table_paid["star_rating"] == '5'].count()
unpaid_5_star = vine_table_unpaid[vine_table_unpaid["star_rating"] == '5'].count()

#Percentage of 5 start reviews 
percentage_paid_5_star = paid_5_star/paid_total_reviews * 100
percentage_unpaid_5_star = unpaid_5_star/unpaid_total_reviews * 100


data = {'paid_total_reviews':[paid_total_reviews], 'unpaid_total_reviews':[unpaid_total_reviews], 
        'paid_5_star':[unpaid_total_reviews], 'unpaid_5_star':[unpaid_5_star],
        'percentage_paid_5_star':[percentage_paid_5_star],'percentage_unpaid_5_star':[percentage_unpaid_5_star]} 

final_df = pd.DataFrame(data)
final_df.reset_index(drop=True, inplace=True)
final_df

,paid_total_reviews,unpaid_total_reviews,paid_5_star,unpaid_5_star,percentage_paid_5_star,percentage_unpaid_5_star
0,3,3094,3094,1704,66.666667,55.074337
